In [2]:
import numpy as np
import numpy.linalg as la
import sympy as sp

In [3]:
def gradient(formula, symbols):
    '''
    Given a SymPy formula and variables
    Find its analytic gradient without substituion
    as a list of SymPy formulae
    '''
    size = len(symbols)
    gradient = []
    for i in range(size):
        gradient.append(formula.diff(symbols[i]))
    return gradient

In [4]:
def gradient_at(gradient, symbols, values):
    '''
    Given a SymPy analytic gradient, variables
    and the values (as list) to substitue
    Return its float point value as a list
    '''
    # Make sure you don't mess up the analytical gradient
    g_copy = gradient.copy()
    gradient_at = []
    for i in range(len(g_copy)):
        for j in range(len(symbols)):
            g_copy[i] = g_copy[i].subs(symbols[j], values[j])
        gradient_at.append(float(g_copy[i].evalf()))
    return gradient_at

In [5]:
def hessian(gradient, symbols):
    '''
    Given an analytic gradient and variables
    Calculate its analytic Hessian
    '''
    size = len(symbols)
    hessian = []
    for i in range(size):
        hessian.append([0]*size)
    for i in range(size):
        for j in range(size):
            hessian[i][j] = gradient[i].diff(symbols[j])
    return hessian

In [6]:
def hessian_at(hessian, symbols, values):
    '''
    Given an analytic Hessian
    Calculate its value with the given substitutions
    '''
    hessian_at = hessian.copy()
    size = len(hessian)
    for i in range(size):
        for j in range(size):
            for k in range(len(symbols)):
                hessian_at[i][j] = hessian_at[i][j].subs(symbols[k], values[k])
            hessian_at[i][j] = float(hessian_at[i][j].evalf())
    return hessian_at

In [7]:
def newton_nd_optimization_crude_step(formula, symbols, x_prev):
    x_prev = list(x_prev)
    grad = gradient(formula, symbols)
    neg_gradient_at = -1 * np.array(gradient_at(grad, symbols, x_prev))
    hes_at = np.array(hessian_at(hessian(grad, symbols), symbols, x_prev))
    return np.array(x_prev) + la.solve(hes_at, neg_gradient_at)

In [8]:
def newton_nd_optimization_crude(f_str, s_str, start, tolerance, actual_solution):
    '''
    A crude version of Newton ND Optimization algorithm
    Maybe you can help out implementing an analytic solution
    Returning the numerical solution as well as the number of
    iterations it took
    '''
    formula = sp.sympify(f_str)
    symbols = sp.symbols(s_str)
    curr = np.copy(start)
    iterations = 0
    while (la.norm(curr - actual_solution, 2) > tolerance):
        curr = newton_nd_optimization_crude_step(formula, symbols, curr)
        iterations += 1
    return curr, iterations

In [17]:
# Test case set-up
form = sp.sympify('12*x**2+10*x*y+12*y**2+8*E**(9*x*y)+8*(sin(y)**2)+9*cos(x*y)') # 记住，大E才是natural number
x, y = sp.symbols('x y')
tolerance = 10**-7
# Test Case: Gradient
grad = gradient(form, (x, y))
grad_at = gradient_at(grad, (x, y), [0, 1])
expected = np.array([
    float(form.diff(x).subs(x, 0).subs(y, 1).evalf()),
    float(form.diff(y).subs(x, 0).subs(y, 1).evalf())
])
actual = np.array(grad_at)
assert la.norm(expected - actual, 2) < tolerance

In [19]:
# Test Case: Hessian
hes = hessian(grad, (x, y))
hes_at = hessian_at(hes, (x, y), [0, 1])
expected = np.array([
    [
        float(form.diff(x).diff(x).subs(x, 0).subs(y, 1).evalf()),
        float(form.diff(x).diff(y).subs(x, 0).subs(y, 1).evalf())
    ], [
        float(form.diff(y).diff(x).subs(x, 0).subs(y, 1).evalf()),
        float(form.diff(y).diff(y).subs(x, 0).subs(y, 1).evalf())
    ]
])
actual = np.array(hes_at)
assert la.norm(expected - actual, 2) < tolerance

In [20]:
# Test Case: Newton ND step
H = np.array([
    [663, 82],
    [82, -16*np.sin(1)**2 + 16*np.cos(1)**2 + 24]
])
v = np.array([82, 16*np.sin(1)*np.cos(1) + 24])
expected = la.solve(H, v) * -1 + np.array([0, 1])
actual = newton_nd_optimization_crude_step(form, (x, y), np.array([0, 1]))
assert la.norm(expected - actual, 2) < tolerance